<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=5f9f759557f4663db72fc52c8ab2b59d4b4ee366f3881abce93c3ea90d2708d9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 16:11:51
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.39 C
-------------------
Today PnL: -2.39 L (-1.7%)
Current PnL: -29.40 L (-18.99%)
CY Booked + Current PnL: -15.11 L (-9.76%)
-------------------
Total profit:  1.25 L
Total loss:  -30.64 L
-------------------
Total Booked + Current PnL: 11.87 L (9.29%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.29%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 94.03 L (67.7%)
Deployed:  1.28 C
Current:  1.39 C
CAGR/XIRR %: 4.67%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-1.23,-17.64,21.54,0.09,17877.0,-17782.0,82995.0,92.53,33.0,M-SC,2.33,253.0,-0.99,0.62,9.04,OX40N,NTT,DURABLES
43,ITC,452.00,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-2.57,-14.49,16.96,0.02,25316.0,-25288.0,149271.0,127.68,47.0,M-SC,16.75,234.0,-1.00,1.11,29.84,OX40N,NTT,PAINTS
50,MASFIN,398.61,-2.98,-4.76,28.15,22.05,26268.0,-4665.0,93315.0,-17.80,50.0,H-SC,6.79,164.0,-0.18,0.70,35.64,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.67,4.62,14.88,20.19,27138.0,8062.0,182382.0,-17.91,43.0,X-MC,6.16,68.0,0.30,1.36,20.44,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.42,-31.04,107.13,42.84,48025.0,-20177.0,44829.0,-54.82,37.0,X-SC,37.24,83.0,-0.42,0.33,1.42,XY24,NTT,MISC
24,EASEMYTRIP,26.40,1.30,-56.95,238.46,45.70,181497.0,-100703.0,76112.0,-20.98,51.0,M-SC,23.51,205.0,-0.55,0.57,9.70,XY24,NTT,TRAVEL
54,PGHH,17907.65,1.10,-4.94,40.72,33.76,77730.0,-9930.0,190890.0,-32.91,38.0,X-MC,5.01,57.0,-0.13,1.43,1.10,X40,ATH,FMCG
68,STARHEALTH,761.00,0.84,-10.66,64.83,47.26,162830.0,-29958.0,251165.0,21.92,34.0,H-SC,13.77,171.0,-0.18,1.88,34.57,XY24,NTT,INSURANCE
84,WIPRO,420.00,0.57,7.21,60.69,72.27,110249.0,12211.0,181659.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.36,13.83,XR,NTT,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-6.72,-20.54,84.93,46.95,84561.0,-25732.0,99565.0,-26.70,26.0,H-SC,4.25,154.0,-0.30,0.74,24.73,MH,ATH,POWER
44,JCHAC,2282.00,-5.31,-38.87,63.56,-0.01,49660.0,-49674.0,78131.0,15878.26,5.0,M-SC,2.41,233.0,-1.00,0.58,0.00,OX40N,NTT,AC
57,RAJOOENG,143.10,-4.49,-35.01,114.83,39.61,76488.0,-35890.0,66610.0,-56.03,26.0,H-SC,25.19,135.0,-0.47,0.50,0.00,AR,ATH,MISC
18,COFFEEDAY,80.00,-4.29,-46.04,150.71,35.27,92335.0,-52282.0,61267.0,-57.75,21.0,L-SC,12.91,270.0,-0.57,0.46,49.95,XR,NTT,HOTELS
0,5PAISA,593.00,-3.85,-42.43,96.13,12.90,120910.0,-92718.0,125778.0,85.10,39.0,H-SC,11.51,161.0,-0.77,0.94,4.58,OX40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.63,-9.16,128.88,107.91,172849.0,-13528.0,134116.0,-26.23,24.0,M-SC,11.62,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.57,-14.49,16.96,0.02,25316.0,-25288.0,149271.0,127.68,47.0,M-SC,16.75,234.0,-1.00,1.11,29.84,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.23,-17.64,21.54,0.09,17877.0,-17782.0,82995.0,92.53,33.0,M-SC,2.33,253.0,-0.99,0.62,9.04,OX40N,NTT,DURABLES
66,SIS,528.00,-0.99,-25.18,62.06,21.26,51559.0,-27953.0,83079.0,1958.55,44.0,H-SC,4.22,166.0,-0.54,0.62,12.25,OX40N,NTT,MISC
47,KANSAINER,340.00,-2.74,-25.96,53.26,13.47,106342.0,-70002.0,199665.0,-69.48,25.0,H-SC,2.90,158.0,-0.66,1.49,1.52,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.57,7.21,60.69,72.27,110249.0,12211.0,181659.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.36,13.83,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.57,7.21,60.69,72.27,110249.0,12211.0,181659.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.36,13.83,XR,NTT,IT
38,INDIAMART,4810.62,-1.69,-2.31,111.60,106.72,134467.0,-2846.0,120490.0,-51.58,35.0,H-SC,1.32,139.0,-0.02,0.90,20.53,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.74,-2.50,39.11,35.63,79916.0,-5237.0,204337.0,-16.30,37.0,H-MC,3.18,119.0,-0.07,1.53,13.76,AR,ATH,PHARMA
37,IEX,219.00,-2.34,-3.69,54.32,48.63,104605.0,-7382.0,192572.0,-36.00,49.0,H-SC,14.13,136.0,-0.07,1.44,7.43,XR,NTT,MISC
25,FINCABLES,1641.55,-2.63,-9.16,128.88,107.91,172849.0,-13528.0,134116.0,-26.23,24.0,M-SC,11.62,216.0,-0.08,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.99,-23.40,93.81,48.45,144079.0,-46920.0,153586.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.15,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.91,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,23.0,X-SC,14.97,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.35,-12.90,39.58,21.58,48314.0,-18072.0,122067.0,-56.10,24.0,X-MC,5.36,56.0,-0.37,0.91,8.07,X40N,ATH,IT
46,JSWINFRA,342.00,-2.34,-11.41,30.06,15.23,53354.0,-22856.0,177491.0,-24.28,26.0,X-MC,7.57,66.0,-0.43,1.33,17.16,X40N,NTT,REALTY
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.87,-20.69,93.54,53.50,223863.0,-62428.0,239323.0,-61.59,34.0,X-MC,1.36,58.0,-0.28,1.79,6.30,XY24,NTT,FMCG
43,ITC,452.0,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG
3,ACC,3906.0,-0.12,-24.52,117.48,64.15,210994.0,-58351.0,179600.0,-55.88,37.0,X-SC,1.68,82.0,-0.28,1.34,0.75,XY24,BTT,CEMENT
15,CAMS,4762.0,-2.88,-80.26,532.07,24.79,264566.0,-202132.0,49724.0,-81.09,41.0,X-SC,2.10,86.0,-0.76,0.37,21.56,X40N,NTT,MISC
36,ICICIPRULI,790.0,-1.57,2.16,28.19,30.97,50900.0,3823.0,180561.0,-21.83,51.0,X-MC,2.16,75.0,0.08,1.35,15.00,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.91,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,23.0,X-SC,14.97,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-1.99,-23.40,93.81,48.45,144079.0,-46920.0,153586.0,-28.89,10.0,X-MC,10.65,64.0,-0.33,1.15,0.00,X40N,ATH,FINANCE
53,PAGEIND,51605.07,-0.59,-9.04,38.59,26.07,57476.0,-14800.0,148940.0,-33.02,30.0,X-MC,11.18,55.0,-0.26,1.11,0.00,X40,ATH,APPARELS
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
59,RELAXO,1176.00,-1.64,-48.47,195.59,52.31,146293.0,-70364.0,74796.0,-45.12,31.0,X-SC,5.77,91.0,-0.48,0.56,0.12,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,1.42,-31.04,107.13,42.84,48025.0,-20177.0,44829.0,-54.82,37.0,X-SC,37.24,83.0,-0.42,0.33,1.42,XY24,NTT,MISC
15,CAMS,4762.00,-2.88,-80.26,532.07,24.79,264566.0,-202132.0,49724.0,-81.09,41.0,X-SC,2.10,86.0,-0.76,0.37,21.56,X40N,NTT,MISC
59,RELAXO,1176.00,-1.64,-48.47,195.59,52.31,146293.0,-70364.0,74796.0,-45.12,31.0,X-SC,5.77,91.0,-0.48,0.56,0.12,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.91,-38.82,120.98,35.20,95243.0,-49944.0,78726.0,3.25,23.0,X-SC,14.97,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.35,-12.90,39.58,21.58,48314.0,-18072.0,122067.0,-56.10,24.0,X-MC,5.36,56.0,-0.37,0.91,8.07,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-0.05,-12.06,35.64,19.28,108428.0,-41725.0,304231.0,-23.30,70.0,X-LC,6.05,1.0,-0.38,2.27,12.47,X40,ATH,IT
41,INFY,2275.00,-0.33,9.66,41.23,54.87,141460.0,30212.0,343100.0,-14.57,68.0,X-LC,3.38,2.0,0.21,2.56,18.74,X40,BTT,IT
75,TMPV,600.00,-1.50,-17.46,72.27,42.18,164874.0,-48274.0,228136.0,-25.52,27.0,X-LC,4.53,3.0,-0.29,1.70,0.00,XY24,NTT,AUTO
81,VBL,671.64,-2.11,-5.10,42.96,35.67,128765.0,-16110.0,299732.0,-16.39,52.0,X-LC,4.92,4.0,-0.13,2.24,8.05,X40N,ATH,FMCG
43,ITC,452.00,-0.65,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,45.0,X-LC,1.60,5.0,-0.14,1.47,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-2.71,-19.45,125.55,81.67,94867.0,-18249.0,75561.0,6788.89,32.0,L-SC,15.48,271.0,-0.19,0.56,46.33,XR,NTT,CERAMICS
50,MASFIN,398.61,-2.98,-4.76,28.15,22.05,26268.0,-4665.0,93315.0,-17.80,50.0,H-SC,6.79,164.0,-0.18,0.70,35.64,XR,ATH,FINANCE
13,BSOFT,831.70,-1.06,-22.38,94.25,50.79,102900.0,-31475.0,109178.0,-0.27,69.0,H-SC,5.56,151.0,-0.31,0.82,27.12,XR,ATH,IT
39,INDIGOPNTS,1408.00,-2.57,-14.49,16.96,0.02,25316.0,-25288.0,149271.0,127.68,47.0,M-SC,16.75,234.0,-1.00,1.11,29.84,OX40N,NTT,PAINTS
68,STARHEALTH,761.00,0.84,-10.66,64.83,47.26,162830.0,-29958.0,251165.0,21.92,34.0,H-SC,13.77,171.0,-0.18,1.88,34.57,XY24,NTT,INSURANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-1.06,-22.38,94.25,50.79,102900.0,-31475.0,109178.0,-0.27,69.0,H-SC,5.56,151.0,-0.31,0.82,27.12,XR,ATH,IT
5,ASIANPAINT,3460.25,-1.35,2.35,18.17,20.95,46822.0,5922.0,257690.0,1.34,70.0,X-LC,18.87,31.0,0.13,1.92,38.83,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.33,9.58,13.82,24.73,36638.0,23186.0,265110.0,14.46,72.0,X-LC,5.55,7.0,0.63,1.98,26.28,X40,ATH,IT
84,WIPRO,420.00,0.57,7.21,60.69,72.27,110249.0,12211.0,181659.0,-7.89,74.0,M-LC,2.79,99.0,0.11,1.36,13.83,XR,NTT,IT
73,TCS,4389.97,-0.05,-12.06,35.64,19.28,108428.0,-41725.0,304231.0,-23.30,70.0,X-LC,6.05,1.0,-0.38,2.27,12.47,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.93
1,25,45.90
2,50,76.78


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.76
MC    30.33
LC    24.94
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.27
X40      22.67
X40N     10.86
XR       10.02
XY25      9.38
AR        9.37
OX40N     7.71
SR        1.01
MH        0.74
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.35
X-MC    23.61
X-LC    21.34
M-SC    12.07
X-SC     6.96
H-MC     4.95
M-MC     1.41
L-SC     1.38
M-LC     1.36
H-LC     1.20
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.48,-7.54,42.31
IT,12.91,-16.51,79.50
FINANCE,9.53,-21.69,73.10
ELECTRICAL,6.24,-12.34,53.48
PAINTS,6.15,-11.92,28.55
MISC,6.14,-57.89,122.57
INSURANCE,4.59,-2.94,39.22
PHARMA,4.12,-1.31,33.73
AUTO,3.43,-25.25,78.95


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3252236.0,21
XR,1321642.0,13
AR,1320298.0,10
X40,1002063.0,14
X40N,994900.0,9
OX40N,757354.0,10
XY25,384534.0,6
SR,285081.0,2
MH,84561.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3719742.0,25
M-SC,1442167.0,15
X-MC,1322880.0,16
X-SC,986971.0,8
X-LC,903453.0,11
H-MC,416098.0,3
L-SC,276476.0,3
M-LC,110249.0,1
H-LC,73360.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1308597.0      6
           AR         901837.0      5
M-SC       XY24       817100.0      6
H-SC       XR         793249.0      7
X-SC       X40N       507293.0      3
X-MC       X40        458321.0      7
           XY24       392745.0      3
X-LC       X40        357644.0      5
H-SC       OX40N      346417.0      4
M-SC       OX40N      321663.0      5
X-SC       XY24       293580.0      3
H-SC       SR         285081.0      2
X-MC       X40N       282650.0      4
X-LC       XY24       242114.0      2
H-MC       AR         217998.0      2
X-LC       X40N       204957.0      2
H-MC       XY24       198100.0      1
X-MC       XY25       189164.0      2
L-SC       XR         187202.0      2
X-SC       X40        186098.0      2
M-SC       XR         176301.0      2
           AR         127103.0      2
M-LC       XR         110249.0      1
X-LC       XY25        98738.0      2
L-SC       OX40N       89274.0      1
H-SC       MH          84561.0      1
H-LC       AR          73360.0      1
M-MC       XY25        56868.0      1
L-MC       XR          54641.0      1
L-LC       XY25        39764.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
